# Shannon's Demon: Basic Demonstration

This notebook provides a basic introduction to Shannon's Demon rebalancing strategy using synthetic data.

**📚 Learning Path**: This is Part 1 of the tutorial series:
- **01_basic_demo.ipynb** ← You are here (Basic concepts)
- **02_real_data.ipynb** - Real market data analysis
- **03_ml_enhanced.ipynb** - ML-enhanced strategies
- **../demon.ipynb** - Comprehensive analysis (must-see!)

**🎯 Quick Start**: For immediate results, run `python scripts/shannon_demon.py`

## 1. Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Random seed for reproducibility
np.random.seed(42)

## 2. Understanding Shannon's Demon

Shannon's Demon is a rebalancing strategy that:
1. Maintains a fixed allocation between two assets (typically 50/50)
2. Periodically rebalances to restore the target allocation
3. Profits from volatility even when the underlying asset has zero expected return

## 3. Create Synthetic Price Data

We'll create a geometric random walk where the asset has equal probability of going up 10% or down 9.09%.

In [ ]:
def create_random_walk(n_periods=252, initial_price=100, up_factor=1.1, down_factor=0.909):
    """
    Create a geometric random walk.
    
    Parameters:
    - n_periods: Number of time periods
    - initial_price: Starting price
    - up_factor: Multiplier for up moves
    - down_factor: Multiplier for down moves
    
    Returns:
    - prices: Array of prices
    """
    prices = [initial_price]
    
    for _ in range(n_periods):
        if np.random.random() > 0.5:
            prices.append(prices[-1] * up_factor)
        else:
            prices.append(prices[-1] * down_factor)
    
    return np.array(prices)

In [ ]:
# Generate price data
n_years = 5
n_periods = n_years * 252  # Daily data
prices = create_random_walk(n_periods)

# Create dates
dates = pd.date_range(start='2019-01-01', periods=n_periods + 1, freq='D')

# Create DataFrame
df = pd.DataFrame({
    'date': dates,
    'price': prices
})

# Plot the price series
plt.figure(figsize=(12, 6))
plt.plot(df['date'], df['price'])
plt.title('Synthetic Asset Price (Geometric Random Walk)')
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Starting price: ${df['price'].iloc[0]:.2f}")
print(f"Ending price: ${df['price'].iloc[-1]:.2f}")
print(f"Total return: {(df['price'].iloc[-1] / df['price'].iloc[0] - 1) * 100:.2f}%")

## 4. Implement Buy-and-Hold Strategy

In [ ]:
def buy_and_hold(prices, initial_capital=10000, allocation=0.5):
    """
    Implement buy-and-hold strategy.
    
    Parameters:
    - prices: Array of asset prices
    - initial_capital: Starting capital
    - allocation: Fraction allocated to risky asset
    
    Returns:
    - portfolio_values: Array of portfolio values over time
    """
    # Initial allocation
    risky_allocation = initial_capital * allocation
    safe_allocation = initial_capital * (1 - allocation)
    
    # Number of shares
    n_shares = risky_allocation / prices[0]
    
    # Calculate portfolio value over time
    portfolio_values = []
    for price in prices:
        risky_value = n_shares * price
        total_value = risky_value + safe_allocation
        portfolio_values.append(total_value)
    
    return np.array(portfolio_values)

In [ ]:
# Run buy-and-hold strategy
bh_values = buy_and_hold(df['price'].values)

print(f"Buy-and-Hold Final Value: ${bh_values[-1]:,.2f}")
print(f"Buy-and-Hold Return: {(bh_values[-1] / bh_values[0] - 1) * 100:.2f}%")

## 5. Implement Shannon's Demon Strategy

In [ ]:
def shannons_demon(prices, initial_capital=10000, target_allocation=0.5, 
                   rebalance_frequency=20, transaction_cost=0.001):
    """
    Implement Shannon's Demon rebalancing strategy.
    
    Parameters:
    - prices: Array of asset prices
    - initial_capital: Starting capital
    - target_allocation: Target allocation to risky asset
    - rebalance_frequency: Rebalance every N periods
    - transaction_cost: Transaction cost as fraction
    
    Returns:
    - portfolio_values: Array of portfolio values
    - rebalance_dates: List of rebalancing dates
    - trades: List of trades executed
    """
    # Initialize portfolio
    risky_value = initial_capital * target_allocation
    safe_value = initial_capital * (1 - target_allocation)
    n_shares = risky_value / prices[0]
    
    portfolio_values = []
    rebalance_dates = []
    trades = []
    
    for i, price in enumerate(prices):
        # Current values
        risky_value = n_shares * price
        total_value = risky_value + safe_value
        portfolio_values.append(total_value)
        
        # Check if rebalancing needed
        if i > 0 and i % rebalance_frequency == 0:
            # Calculate current allocation
            current_allocation = risky_value / total_value
            
            # Rebalance to target
            target_risky_value = total_value * target_allocation
            target_safe_value = total_value * (1 - target_allocation)
            
            # Calculate trade
            risky_trade_value = target_risky_value - risky_value
            shares_traded = risky_trade_value / price
            
            # Apply transaction costs
            cost = abs(risky_trade_value) * transaction_cost
            
            # Update portfolio
            n_shares += shares_traded
            risky_value = n_shares * price
            safe_value = target_safe_value - cost
            
            # Record trade
            rebalance_dates.append(i)
            trades.append({
                'period': i,
                'price': price,
                'shares_traded': shares_traded,
                'value_traded': risky_trade_value,
                'cost': cost,
                'allocation_before': current_allocation,
                'allocation_after': target_allocation
            })
    
    return np.array(portfolio_values), rebalance_dates, trades

In [ ]:
# Run Shannon's Demon strategy
sd_values, rebalance_dates, trades = shannons_demon(df['price'].values)

print(f"Shannon's Demon Final Value: ${sd_values[-1]:,.2f}")
print(f"Shannon's Demon Return: {(sd_values[-1] / sd_values[0] - 1) * 100:.2f}%")
print(f"Number of rebalances: {len(rebalance_dates)}")
print(f"Total transaction costs: ${sum(t['cost'] for t in trades):,.2f}")

## 6. Compare Strategies

In [ ]:
# Plot comparison
plt.figure(figsize=(14, 8))

# Portfolio values
plt.subplot(2, 1, 1)
plt.plot(df['date'], bh_values, label='Buy and Hold', linewidth=2)
plt.plot(df['date'], sd_values, label="Shannon's Demon", linewidth=2)

# Mark rebalancing points
for rd in rebalance_dates:
    plt.axvline(df['date'].iloc[rd], color='red', alpha=0.3, linestyle='--')

plt.title('Portfolio Value Comparison')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.grid(True, alpha=0.3)

# Returns
plt.subplot(2, 1, 2)
bh_returns = (bh_values[1:] / bh_values[:-1] - 1) * 100
sd_returns = (sd_values[1:] / sd_values[:-1] - 1) * 100

plt.plot(df['date'][1:], np.cumprod(1 + bh_returns/100) - 1, label='Buy and Hold')
plt.plot(df['date'][1:], np.cumprod(1 + sd_returns/100) - 1, label="Shannon's Demon")
plt.title('Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Performance Analysis

In [ ]:
def calculate_metrics(values, periods_per_year=252):
    """
    Calculate performance metrics.
    """
    returns = values[1:] / values[:-1] - 1
    
    # Basic metrics
    total_return = (values[-1] / values[0] - 1) * 100
    annual_return = (values[-1] / values[0]) ** (periods_per_year / len(values)) - 1
    
    # Risk metrics
    volatility = np.std(returns) * np.sqrt(periods_per_year)
    sharpe_ratio = (annual_return - 0.02) / volatility  # Assuming 2% risk-free rate
    
    # Drawdown
    cumulative = np.cumprod(1 + returns)
    running_max = np.maximum.accumulate(cumulative)
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = np.min(drawdown) * 100
    
    return {
        'total_return': total_return,
        'annual_return': annual_return * 100,
        'volatility': volatility * 100,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown
    }

In [ ]:
# Calculate metrics for both strategies
bh_metrics = calculate_metrics(bh_values)
sd_metrics = calculate_metrics(sd_values)

# Create comparison DataFrame
metrics_df = pd.DataFrame({
    'Buy and Hold': bh_metrics,
    "Shannon's Demon": sd_metrics
}).T

print("Performance Metrics Comparison:")
print("=" * 50)
print(metrics_df.round(2))

## 8. Rebalancing Analysis

In [ ]:
# Analyze trades
trades_df = pd.DataFrame(trades)

if len(trades_df) > 0:
    # Trade statistics
    print("\nRebalancing Statistics:")
    print("=" * 50)
    print(f"Total trades: {len(trades_df)}")
    print(f"Average trade size: ${abs(trades_df['value_traded']).mean():,.2f}")
    print(f"Total transaction costs: ${trades_df['cost'].sum():,.2f}")
    print(f"Cost as % of final value: {trades_df['cost'].sum() / sd_values[-1] * 100:.2f}%")
    
    # Plot allocation drift
    plt.figure(figsize=(12, 6))
    
    allocation_before = trades_df['allocation_before'].values
    trade_periods = trades_df['period'].values
    
    plt.scatter(trade_periods, allocation_before * 100, color='red', alpha=0.6, label='Before rebalance')
    plt.axhline(50, color='green', linestyle='--', label='Target allocation')
    
    plt.title('Portfolio Allocation Drift')
    plt.xlabel('Period')
    plt.ylabel('Risky Asset Allocation (%)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

## 9. Sensitivity Analysis

In [ ]:
# Test different rebalancing frequencies
frequencies = [5, 10, 20, 40, 60, 120]
results = []

for freq in frequencies:
    values, _, trades = shannons_demon(df['price'].values, rebalance_frequency=freq)
    metrics = calculate_metrics(values)
    
    results.append({
        'frequency': freq,
        'total_return': metrics['total_return'],
        'sharpe_ratio': metrics['sharpe_ratio'],
        'num_trades': len(trades),
        'total_costs': sum(t['cost'] for t in trades) if trades else 0
    })

sensitivity_df = pd.DataFrame(results)

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Total return vs frequency
axes[0, 0].plot(sensitivity_df['frequency'], sensitivity_df['total_return'], 'o-')
axes[0, 0].set_xlabel('Rebalancing Frequency (days)')
axes[0, 0].set_ylabel('Total Return (%)')
axes[0, 0].set_title('Return vs Rebalancing Frequency')
axes[0, 0].grid(True, alpha=0.3)

# Sharpe ratio vs frequency
axes[0, 1].plot(sensitivity_df['frequency'], sensitivity_df['sharpe_ratio'], 'o-', color='orange')
axes[0, 1].set_xlabel('Rebalancing Frequency (days)')
axes[0, 1].set_ylabel('Sharpe Ratio')
axes[0, 1].set_title('Sharpe Ratio vs Rebalancing Frequency')
axes[0, 1].grid(True, alpha=0.3)

# Number of trades
axes[1, 0].bar(sensitivity_df['frequency'], sensitivity_df['num_trades'], color='green', alpha=0.7)
axes[1, 0].set_xlabel('Rebalancing Frequency (days)')
axes[1, 0].set_ylabel('Number of Trades')
axes[1, 0].set_title('Trading Activity')
axes[1, 0].grid(True, alpha=0.3)

# Transaction costs
axes[1, 1].bar(sensitivity_df['frequency'], sensitivity_df['total_costs'], color='red', alpha=0.7)
axes[1, 1].set_xlabel('Rebalancing Frequency (days)')
axes[1, 1].set_ylabel('Total Transaction Costs ($)')
axes[1, 1].set_title('Transaction Costs')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nSensitivity Analysis Results:")
print("=" * 60)
print(sensitivity_df.to_string(index=False))

## 10. Key Takeaways

1. **Volatility Harvesting**: Shannon's Demon profits from volatility, not directional moves
2. **Rebalancing Frequency**: More frequent rebalancing can improve returns but increases costs
3. **Transaction Costs**: Critical factor in determining optimal rebalancing frequency
4. **Risk-Adjusted Returns**: Often provides better Sharpe ratios than buy-and-hold

### Next Steps

- Try the strategy with real market data (see notebook 02)
- Implement threshold-based rebalancing
- Add machine learning to predict optimal rebalancing times (see notebook 03)
- Test with different asset pairs and correlations

In [ ]:
# Save results for further analysis
results_summary = {
    'buy_and_hold': {
        'final_value': bh_values[-1],
        'total_return': bh_metrics['total_return'],
        'sharpe_ratio': bh_metrics['sharpe_ratio']
    },
    'shannons_demon': {
        'final_value': sd_values[-1],
        'total_return': sd_metrics['total_return'],
        'sharpe_ratio': sd_metrics['sharpe_ratio'],
        'num_trades': len(trades),
        'total_costs': sum(t['cost'] for t in trades)
    }
}

print("\nExperiment completed successfully!")
print(f"Shannon's Demon outperformance: {sd_metrics['total_return'] - bh_metrics['total_return']:.2f}%")